In [12]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
steam_games = pd.read_csv('steam_tags_with_pics_v2.csv', delimiter='\t')
steam_games = steam_games.drop(['Unnamed: 0'], axis=1)

In [3]:
app_id = steam_games['steam_url'].str.split('/',expand=True)
app_id = app_id.drop([0, 1, 2, 5, 6], axis=1)
app_id.columns = ['app', 'game_ID']
#app_id_final = app_id_2[app_id_2[4]] 

In [4]:
steam_games_id = pd.concat([steam_games, app_id], axis=1, sort=False)

In [5]:
steam_games_id.head()

,title,steam_url,game_image,steam_tags,app,game_ID
0,Counter-Strike: Global Offensive,https://store.steampowered.com/app/730/Counter...,https://steamcdn-a.akamaihd.net/steam/apps/730...,"FPS, Multiplayer, Shooter, Action, Team-Based,...",app,730
1,MONSTER HUNTER: WORLD,https://store.steampowered.com/app/582010/MONS...,https://steamcdn-a.akamaihd.net/steam/apps/582...,"Action, Hunting, Co-op, Multiplayer, Open Worl...",app,582010
2,Path of Exile,https://store.steampowered.com/app/238960/Path...,https://steamcdn-a.akamaihd.net/steam/apps/238...,"Free to Play, Action RPG, RPG, Hack and Slash,...",app,238960
3,Insurgency: Sandstorm,https://store.steampowered.com/app/581320/Insu...,https://steamcdn-a.akamaihd.net/steam/apps/581...,"FPS, Realistic, Action, Shooter, Multiplayer, ...",app,581320
4,Warframe,https://store.steampowered.com/app/230410/Warf...,https://steamcdn-a.akamaihd.net/steam/apps/230...,"Free to Play, Action, Co-op, Multiplayer, Thir...",app,230410


In [6]:
steam_games_id = steam_games_id[~steam_games_id.app.str.contains('sub')]
steam_games_id = steam_games_id[~steam_games_id.app.str.contains('digitalgiftcards')]
steam_games_id = steam_games_id[~steam_games_id.steam_tags.str.contains('no_tag')]
steam_games_id = steam_games_id.drop(['app'], axis=1)

In [7]:
steam_games_id['game_ID'] = pd.to_numeric(steam_games_id['game_ID'], downcast='signed')

In [8]:
steam_games_id.dtypes

title         object
steam_url     object
game_image    object
steam_tags    object
game_ID        int32
dtype: object

In [9]:
steam_games_id=steam_games_id.rename(columns={"game_ID": "appid"})
steam_games_id.head()

,title,steam_url,game_image,steam_tags,appid
0,Counter-Strike: Global Offensive,https://store.steampowered.com/app/730/Counter...,https://steamcdn-a.akamaihd.net/steam/apps/730...,"FPS, Multiplayer, Shooter, Action, Team-Based,...",730
1,MONSTER HUNTER: WORLD,https://store.steampowered.com/app/582010/MONS...,https://steamcdn-a.akamaihd.net/steam/apps/582...,"Action, Hunting, Co-op, Multiplayer, Open Worl...",582010
2,Path of Exile,https://store.steampowered.com/app/238960/Path...,https://steamcdn-a.akamaihd.net/steam/apps/238...,"Free to Play, Action RPG, RPG, Hack and Slash,...",238960
3,Insurgency: Sandstorm,https://store.steampowered.com/app/581320/Insu...,https://steamcdn-a.akamaihd.net/steam/apps/581...,"FPS, Realistic, Action, Shooter, Multiplayer, ...",581320
4,Warframe,https://store.steampowered.com/app/230410/Warf...,https://steamcdn-a.akamaihd.net/steam/apps/230...,"Free to Play, Action, Co-op, Multiplayer, Thir...",230410


In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(steam_games_id['steam_tags'])
tfidf_matrix.shape

(29274, 384)

In [57]:


cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(steam_games_id.index, index=steam_games_id['title']).drop_duplicates()

In [74]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    game_indices = [i[0] for i in sim_scores]
    return steam_games_id['title'].iloc[game_indices]

In [76]:
get_recommendations('HITMAN™')

844                             Megadimension Neptunia VII
866      Hyperdimension Neptunia Re;Birth1 / 超次次元ゲイム ネプ...
27       DRAGON QUEST® XI: Echoes of an Elusive Age™ - ...
2025            BLUE REFLECTION / BLUE REFLECTION　幻に舞う少女の剣
1165     Superdimension Neptune VS Sega Hard Girls | 超次...
634        The Legend of Heroes: Trails in the Sky the 3rd
1016     Hyperdimension Neptunia Re;Birth3 V Generation...
1192     Hyperdimension Neptunia Re;Birth2: Sisters Gen...
28198                      Serment - Contract with a Devil
398             The Legend of Heroes: Trails in the Sky SC
Name: title, dtype: object

In [ ]:
steam_games_id.to_csv('steam_games_final.csv', sep="\t")

In [14]:
indices = pd.Series(steam_games_id.index, index=steam_games_id['title']).drop_duplicates()

In [13]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(steam_games_id['steam_tags'])
cosine_sim = linear_kernel(count_matrix, count_matrix)

In [15]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    game_indices = [i[0] for i in sim_scores]
    return steam_games_id['title'].iloc[game_indices]

In [16]:
get_recommendations('HITMAN™')

14414                                          Garden Tale
1210                                        Troubleshooter
2326                     Moekuri: Adorable + Tactical SRPG
3345                                           Sproggiwood
3717                   Robothorium: Sci-fi Dungeon Crawler
4317                    FLAMBERGE ||| SIMULTANEOUS TACTICS
956                             Disgaea PC / 魔界戦記ディスガイア PC
1280     Fairy Fencer F Advent Dark Force | フェアリーフェンサー ...
1804                                             EARTHLOCK
2541                             Age of Fear 3: The Legend
Name: title, dtype: object